In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import random
import shutil

from shutil import copyfile

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Dropout, Conv2D, GlobalAveragePooling2D, GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [92]:
base = ''
# MODEL_DESCRIPTION = '3classes_simulated_MobNotTrainable' 
MODEL_DESCRIPTION = '3classes_FineTunning_MobNotTrainable' 

DATAPATH = os.path.join('datasets/Testset/')
MASK_PATH = os.path.join(DATAPATH,'well_ported_mask/')
NO_MASK_PATH = os.path.join(DATAPATH,'no_mask/')
WRONG_MASK_PATH = os.path.join(DATAPATH,'wrong_ported_mask/')
WEIGHTS_PATH = os.path.join(base,'weights/')

initial_weights = os.path.join(WEIGHTS_PATH,'3classes_fineTunning_mobTrainable.h5')
trainable = True

In [93]:
def view(pth):
    images = list()
    for img in random.sample(os.listdir(pth),9):
        images.append(img)
    i = 0
    fig,ax = plt.subplots(nrows=3, ncols=3, figsize=(30,20))
    for row in range(3):
        for col in range(3):
            ax[row,col].imshow(cv2.imread(os.path.join(pth,images[i])))
            i+=1

In [94]:
NumberMaskData = os.listdir(MASK_PATH)
print(len(NumberMaskData))

3


In [95]:
NumberNOMaskData = os.listdir(NO_MASK_PATH)
print(len(NumberNOMaskData))

4


In [96]:
NumberWrongMaskData = os.listdir(WRONG_MASK_PATH)
print(len(NumberWrongMaskData))

5


In [97]:
# view(MASKPATH)

In [98]:
# view(NOMASKPATH)

Analysing our Data

In [99]:
fig = go.Figure(
    data=[go.Pie(labels=['WITHMASK','WITHOUTMASK','WRONGMASK'], 
        values=[NumberMaskData , NumberNOMaskData , NumberWrongMaskData ])
    ])
fig.show()

To clean the NO_MASK and MASK folders

In [100]:
testGen = ImageDataGenerator(
    rescale= 1.0/255.0,
)

Ref : https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

In [101]:
test = testGen.flow_from_directory(
    DATAPATH, 
    target_size=(128, 128),
    classes=['no_mask','well_ported_mask', 'wrong_ported_mask'],
    class_mode='categorical', 
    batch_size=1, 
    shuffle=False,
)


Found 12 images belonging to 3 classes.


In [102]:
mob = MobileNetV2(
    input_shape = (128,128,3),
    include_top = False,
    weights = 'imagenet',
)
mob.trainable = trainable


In [103]:
model = Sequential()
model.add(mob)
model.add(GlobalMaxPool2D())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_max_pooling2d_5 (Glob (None, 1280)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                81984     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 195       
Total params: 2,340,163
Trainable params: 2,306,051
Non-trainable params: 34,112
_________________________________________________________________


In [104]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
model.load_weights( initial_weights )

In [105]:
model.evaluate(test)

12/12 [==============================] - 0s 13ms/step - loss: 0.7640 - accuracy: 0.6667


[0.7639802098274231, 0.6666666865348816]